In [1]:
import spacy
from spacy.tokens import Token
import pickle
import random
import re
from shutil import copy2
import os

In [2]:
train_data = pickle.load(open('train_data.pkl', 'rb'))

In [3]:
train_data[0][1].get('entities')

[(1749, 1755, 'Companies worked at'),
 (1696, 1702, 'Companies worked at'),
 (1417, 1423, 'Companies worked at'),
 (1356, 1793, 'Skills'),
 (1209, 1215, 'Companies worked at'),
 (1136, 1248, 'Skills'),
 (928, 932, 'Graduation Year'),
 (858, 889, 'College Name'),
 (821, 856, 'Degree'),
 (787, 791, 'Graduation Year'),
 (744, 750, 'Companies worked at'),
 (722, 742, 'Designation'),
 (658, 664, 'Companies worked at'),
 (640, 656, 'Designation'),
 (574, 580, 'Companies worked at'),
 (555, 573, 'Designation'),
 (470, 493, 'Companies worked at'),
 (444, 469, 'Designation'),
 (308, 314, 'Companies worked at'),
 (234, 240, 'Companies worked at'),
 (175, 198, 'Companies worked at'),
 (93, 137, 'Email Address'),
 (39, 48, 'Location'),
 (13, 38, 'Designation'),
 (0, 12, 'Name')]

## NER with Spacy 

In [4]:
import warnings

In [5]:
nlp = spacy.blank('en')


def train_model(train_data):
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
   
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        optimizer = nlp.begin_training()
            
        for itn in range(100):
            print('Starting iterations ', str(itn))
            random.shuffle(train_data)
            
            
            losses = {}
            index = 0
            for text, annotations in train_data:
                print(index)
                index = index + 1
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)




In [32]:
train_model(train_data)

c:\users\rathore\appdata\local\programs\python\python37\lib\site-packages\spacy\language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs

c:\users\rathore\appdata\local\programs\python\python37\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Arpit Jain Quality Analyst - ThoughtWorks Technolo..." with entities "[(2301, 2380, 'Skills'), (2088, 2131, 'College Nam...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be igno

Starting iterations  0
0
1


c:\users\rathore\appdata\local\programs\python\python37\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Jatin Arora SDET Automation Engineer, Infosys - CR..." with entities "[(3909, 3931, 'College Name'), (3883, 3908, 'Degre...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)

c:\users\rathore\appdata\local\programs\python\python37\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Srushti Bhadale Mumbai, Maharashtra - Email me on ..." with entities "[(1551, 1852, 'Skills'), (1454, 1499, 'Email Addre...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)



2
3


c:\users\rathore\appdata\local\programs\python\python37\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Arpit Godha Senior Process Executive  Jaipur, Raja..." with entities "[(3144, 3495, 'Skills'), (3081, 3104, 'College Nam...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)

c:\users\rathore\appdata\local\programs\python\python37\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Suresh Kanagala Architecture SharePoint/Office 365..." with entities "[(1171, 1574, 'Skills'), (962, 1096, 'Skills'), (9...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)



...

64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
{'ner': 1600.243783362244}
Starting iterations  46
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137

78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
{'ner': 1201.7161584869202}
Starting iterations  57
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
{'ner': 1168.4887151230623}
Starting iterations  68
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
1

102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
{'ner': 1025.6114969592477}
Starting iterations  79
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
16

112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
{'ner': 985.1058798295703}
Starting iterations  90
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176

In [8]:
nlp.to_disk('nlp_model')

## Model Testing 

In [6]:
nlp_model = spacy.load('nlp_model')

In [7]:
text = train_data[0][0]

In [8]:
doc = nlp_model(text)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Govardhana K
DESIGNATION                   - Senior Software Engineer
LOCATION                      - Bengaluru
DESIGNATION                   - Developer Languages
DESIGNATION                   - Senior Software Engineer
DESIGNATION                   - Senior Consultant
COMPANIES WORKED AT           - Oracle
GRADUATION YEAR               - 2017
COMPANIES WORKED AT           - Oracle
DESIGNATION                   - Associate Consultant
COMPANIES WORKED AT           - Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering
COLLEGE NAME                  - Adithya Institute of Technology
GRADUATION YEAR               - 2012
SKILLS                        - APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)


## Model testing with PDF Data 

In [12]:
!pip install PyMuPDF

In [9]:
import sys, fitz

In [10]:
filename ='test_data/cv1.pdf'
doc = fitz.open(filename)
text = ""
        
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
            
doc = nlp_model(tx)

for ent in doc.ents:
    print(f'{ent.label_.upper():{30}} - {ent.text}')



NAME                           - Alice Clark
DESIGNATION                    - AI / Machine Learning
LOCATION                       - Delhi
DESIGNATION                    - Microsoft Azure cloud services
DESIGNATION                    - Software Engineer
COMPANIES WORKED AT            - Microsoft
GRADUATION YEAR                - 2000
COMPANIES WORKED AT            - Microsoft
COMPANIES WORKED AT            - Microsoft
COMPANIES WORKED AT            - Microsoft
COMPANIES WORKED AT            - Microsoft
COMPANIES WORKED AT            - Microsoft
COMPANIES WORKED AT            - Microsoft
COLLEGE NAME                   - Indian Institute of Technology – Mumbai
GRADUATION YEAR                - 2001
SKILLS                         - Machine Learning, Natural Language Processing, and Big Data Handling    ADDITIONAL INFORMATION  Professional Skills  • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal  skills with ability to interact with individuals at 

In [14]:
directory = 'test_data'

for filename in os.scandir(directory):
    if filename.is_file():
        doc = fitz.open(filename.path)
        text = ""
        
        for page in doc:
            text = text + str(page.getText())
        
        tx = " ".join(text.split('\n'))
        doc = nlp_model(tx)
        
        for token in doc:
            if token.text=="Delhi":
                copy2('/Users/Rathore/Deeksha_task(Part-1)/' + filename.path,'/Users/Rathore/Deeksha_task(Part-1)/Files_based_on_location/Delhi' )
                #print("File copied")
            elif token.text=="Hyderabad":
                copy2('/Users/Rathore/Deeksha_task(Part-1)/' + filename.path,'/Users/Rathore/Deeksha_task(Part-1)/Files_based_on_location/Hyderabad' )
                #print("File copied")
            elif token.text=="Manchester":
                copy2('/Users/Rathore/Deeksha_task(Part-1)/' + filename.path,'/Users/Rathore/Deeksha_task(Part-1)/Files_based_on_location/Manchester' )
                #print("File copied")
            elif token.text=="CA":
                copy2('/Users/Rathore/Deeksha_task(Part-1)/' + filename.path,'/Users/Rathore/Deeksha_task(Part-1)/Files_based_on_location/CA' )
                #print("File copied")
            elif token.text=="Pune":
                copy2('/Users/Rathore/Deeksha_task(Part-1)/' + filename.path,'/Users/Rathore/Deeksha_task(Part-1)/Files_based_on_location/Pune' )
                #print("File copied")

In [14]:
filename ='test_data/cv2.pdf'
doc = fitz.open(filename)
text = ""
        
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
            
doc = nlp_model(tx)

token = doc[9]
if token.text=="Manchester":
    copy2('/Users/Rathore/Deeksha_task/test_data/cv2.pdf','/Users/Rathore/Deeksha_task/Files_based_on_location/Manchester' )
    print("File copied")
else:
    print("Not matching")

File copied


In [15]:
filename ='test_data/cv3.pdf'
doc = fitz.open(filename)
text = ""
        
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
            
doc = nlp_model(tx)

token = doc[1071]
if token.text=="Mumbai":
    copy2('/Users/Rathore/Deeksha_task/test_data/cv3.pdf','/Users/Rathore/Deeksha_task/Files_based_on_location/Mumbai' )
    print("File copied")
else:
    print("Not matching")

File copied


In [16]:
filename ='test_data/cv4.pdf'
doc = fitz.open(filename)
text = ""
        
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
            
doc = nlp_model(tx)

token = doc[16]
if token.text=="Pune":
    copy2('/Users/Rathore/Deeksha_task/test_data/cv4.pdf','/Users/Rathore/Deeksha_task/Files_based_on_location/Pune' )
    print("File copied")
else:
    print("Not matching")

File copied


In [17]:
filename ='test_data/cv5.pdf'
doc = fitz.open(filename)
text = ""
        
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
            
doc = nlp_model(tx)

token = doc[7]
if token.text=="Hyderabad":
    copy2('/Users/Rathore/Deeksha_task/test_data/cv5.pdf','/Users/Rathore/Deeksha_task/Files_based_on_location/Hyderabad' )
    print("File copied")
else:
    print("Not matching")

File copied
